In [1]:
from qTools import *
import Cahit as ck

import scipy.sparse as sp
import numpy as np

from functools import partial
from multiprocessing import Pool, cpu_count

In [2]:
resonatorDimension = 200
g = 1.79
qfreq = 0
resFreq = 2
irreg = False

In [3]:
def digitalRabi(obj, stepSize):
    sigmaX = sp.kron(qOps.identity(obj.subSystems[0].dimension), obj.sigmaX(obj.subSystems[1].dimension), format='csc')
    HamJC = 2 * np.pi * (((obj.subSystems[0].frequency/2)*obj.subSystems[0].freeMat) + obj.couplingHam)
    UnitaryJC = lio.Liouvillian(HamJC, timeStep=(stepSize / 2))
    UnitaryAJC = (UnitaryJC @ UnitaryJC)
    Unitary = UnitaryJC @ sigmaX @ UnitaryAJC @ sigmaX @ UnitaryJC
    return Unitary

In [4]:
JCSys = QuantumSystem()
cav = Cavity(dimension=resonatorDimension, frequency=resFreq)
JCSys.addSubSys(cav)

qub = JCSys.createSubSys(subClass=Qubit)
qub.frequency = qfreq
#qub.freeMat = None

In [5]:
JCcoupling = JCSys.coupleBy(cav, qub,'JC', g)
print(JCcoupling.couplingStrength)
"""JCcoupling = qsys.qCoupling()
JCSys.addSysCoupling(JCcoupling)"""

JCSys.initialState = sp.kron(states.basis(cav.dimension, 0), states.basis(2, 0), format='csc')

sigmaz
1.79


In [6]:
qSim = Simulation(qSys=JCSys)
cavFreqSweep = qSim.addSweep(cav, 'frequency')
print(cavFreqSweep)
cavFreqSweep.sweepList = np.arange(-4, 4, 0.05)
print(cav.freeMat)

cavParity = parityEXP(cav.freeMat)
p = Pool(processes=cpu_count())
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), cavFreqSweep.sweepList)
parityDigit = p.map(partial(expectationKetList, cavParity),statesDigit)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

  (0, 0)	0.0
  (1, 1)	0.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	2.0
  (5, 5)	2.0
  (6, 6)	3.0
  (7, 7)	3.0
  (8, 8)	4.0
  (9, 9)	4.0
  (10, 10)	5.0
  (11, 11)	5.0
  (12, 12)	6.0
  (13, 13)	6.0
  (14, 14)	7.0
  (15, 15)	7.0
  (16, 16)	8.0
  (17, 17)	8.0
  (18, 18)	9.0
  (19, 19)	9.0
  (20, 20)	10.0
  (21, 21)	10.0
  (22, 22)	11.0
  (23, 23)	11.0
  (24, 24)	12.0
  :	:
  (375, 375)	187.0
  (376, 376)	188.0
  (377, 377)	188.0
  (378, 378)	189.0
  (379, 379)	189.0
  (380, 380)	190.0
  (381, 381)	190.0
  (382, 382)	191.0
  (383, 383)	191.0
  (384, 384)	192.0
  (385, 385)	192.0
  (386, 386)	193.0
  (387, 387)	193.0
  (388, 388)	194.0
  (389, 389)	194.0
  (390, 390)	195.0
  (391, 391)	195.0
  (392, 392)	196.0
  (393, 393)	196.0
  (394, 394)	197.0
  (395, 395)	197.0
  (396, 396)	198.0
  (397, 397)	198.0
  (398, 398)	199.0
  (399, 399)	199.0
simulating


NameError: name 'expectationKetList' is not defined

In [ ]:
JCSys.reset()
JCSys.createSysCoupling([qub,cav],[qtbox.operators.sigmax, qtbox.operators.create], g)
JCSys.createSysCoupling([cav, qub],[qtbox.operators.destroy, qtbox.operators.sigmax], g)
JCSys.couplingName = 'Rabi'

print(JCSys.couplingName)

In [ ]:
qSim.qSys = JCSys
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationKetList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='JC')
#JCSys.addCoupling([qub,cav],[qtbox.operators.destroy, qtbox.operators.create], g)
#JCSys.addCoupling([cav, qub],[qtbox.operators.destroy, qtbox.operators.create], g)
JCSys.sigmaX = oper.sigmax
JCSys.Unitaries = digitalRabi

In [ ]:
qSim.qSys = JCSys
print('simulating')
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
print('calculating')
parityDigit = p.map(partial(qFncs.expectationKetList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='Rabi')

qSim.qSys = JCSys

statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationKetList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to='JC')
JCcoupling.couplingStrength = 1
qSim.qSys = JCSys
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationKetList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)

In [ ]:
JCSys.reset(to=2)
print(JCSys.couplingName)
JCcoupling.couplingStrength = 1.79
qSim.qSys = JCSys
statesDigit = p.map(partial(qSim.evolveTimeIndep, cav), qSim.sweepList)
parityDigit = p.map(partial(qFncs.expectationKetList, cavParity),statesDigit)
sPlt.colorPlot(qSim.sweepList, qSim.times, parityDigit, irregular=irreg)
print(JCSys._QuantumSystem__kept)